# Lab 10-4: ImageFolder - 자신의 데이터셋 학습시키기


## Imports

In [5]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

from matplotlib.pyplot import imshow
%matplotlib inline

## ImageFolder Example

In [8]:
trans = transforms.Compose([
                            transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root='custom_data/origin_data',
                                              transform= trans)

for i, value in enumerate(train_data):
  data, label = value
  print(i, data, label)

  if(label==0):
    data.save('custom_data/train_data/gray/{}_{}'.format(i, label))
  else:
    data.save('custom_data/train_data/red/{}_{}'.format(i, label))


FileNotFoundError: ignored

## Use ImageFolder

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#For reproducibility
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

# data & data_loader
trans = transforms.Compose([
                            transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data',
                                              transforms = trans)

data_loader = torch.utils.data.DataLoader(dataset= train_data,
                                          batch_size = 8,
                                          shuffle = True,
                                          num_workers=2)

# Model
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3,6,5),
        nn.ReLU(),
        nn.MaxPool2d()
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d()
    )
    self.layer1 = nn.Sequential(
        nn.Linear(16*13*29, 120),
        nn.ReLU(),
        nn.Linear(120, 2)
    )
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.shape[0], -1)
    out = self.layer3(out)
    return out


NameError: ignored

In [ ]:
# testing
net = CNN().to(device)
test_input = torch.Tensor(3,3,64,128).to(device)
test_out = net(test_input)

In [ ]:
#cost & optimizer
optimizer = torch.optim.Adam(net.parameters(), lr= 0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

# training
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
  avg_cost = 0
  for num, data in enumerate(data_loader):
    X, Y = data
    
    X = X.to(device)
    Y = Y.to(device)

    optimizer.no_grad()
    out = net(X)
    loss = loss_func(out, Y)
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch
  
  priprint('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

In [ ]:
#save model state
torch.save(net.state_dict(), './model/model.pth')

#load model state
new_net = CNN().to(device)
new_net.load_state_dict(torch.load('./model/model.pth'))

# compare
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

In [ ]:
# test
trans = torchvision.transforms.Compose([
                                        transforms.Resize((64, 128))
                                        transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data',
                                             transform= trans)

test_set = torch.utils.data.DataLoader(dataset= test_data,
                                       batch_size= len(test_data))

with torch.no_grad():
  for num,data in enumerate(test_set):
    X, Y = data
    X = X.to(device)
    Y = Y.to(device)

    prediction = net(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy = correct_prediction.float().mean()

    print('Accuracy:', accuracy.item())